In [8]:
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
from sklearn.manifold import TSNE

import numpy as np
import altair as alt
import pandas as pd
import os

In [9]:
# bolt_url = os.getenv("NEO4J_BOLT_URL", "bolt://localhost")
# user = os.getenv("NEO4J_USER", "neo4j")
# password = os.getenv("NEO4J_PASSWORD", "neo")
# driver = GraphDatabase.driver("bolt://graph-embeddings-neo4j", auth=(user, password))

# Connection to Neo4j Db.
graph_name = "neo4j" # Default for 4.0: neo4j; default for 3.5: graph.db
SERVER_ADDRESS = "bolt://localhost:7687"
SERVER_AUTH = ("neo4j", "erclab")
driver = GraphDatabase.driver(SERVER_ADDRESS, auth=SERVER_AUTH)

In [10]:
result = {"label": [], "count": []}
with driver.session(database=graph_name) as session:
    for row in session.run("CALL db.labels()"):
        label = row["label"]
        query = f"MATCH (:`{label}`) RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values("count")

result = {"relType": [], "count": []}
with driver.session(database=graph_name) as session:
    for row in session.run("CALL db.relationshipTypes()"):
        relationship_type = row["relationshipType"]
        query = f"MATCH ()-[:`{relationship_type}`]->() RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["relType"].append(relationship_type)
        result["count"].append(count)
rels_df = pd.DataFrame(data=result)
rels_df.sort_values("count")

display(nodes_df)
display(rels_df)

,label,count
0,Restaurant,80455
1,R_Attribute,8
2,Category,911
3,Photo,194119
4,City,476
5,State,18
6,Country,2
7,User,1968703
8,Review,5508465


,relType,count
0,IN_CATEGORY,338159
1,HAS,515896
2,IN_CITY,80455
3,IN_STATE,476
4,IN_COUNTRY,18
5,FRIENDS,9521050
6,WROTE,5508465
7,REVIEWS_FOR,5508465
